### Load Packages

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
import datetime
import re

warnings.simplefilter("ignore")

pd.options.mode.chained_assignment = None

### Import Excel Files

In [ ]:
folder = r'C:\Users\mazhang\Documents\Jupyter Notebook\Monthly Closed Invoice\Monthly Storage Income\2022'
files = os.listdir(folder)

excel_list = []
for file in files:
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder, file), sheet_name = None)
        excel_list.append(df)

In [ ]:
df_list = []
# month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

month = ['Jan', 'Feb', 'Mar']

for excel in range(len(excel_list)):
    
    df = excel_list[excel]['Sheet1']
    df = df.fillna('N/A')

    df1 = df.groupby(['WISE Facility', 'Customer', 'Accounting_CustomerID'])['Amount'].sum().reset_index()
    df2 = df1.rename(columns = {'Amount': month[excel]})
    
    print(df2.shape)
    
    df_list.append(df2)

In [ ]:
df_list[-1]

In [ ]:
df_combine_list = []
df_combine_year = df_list[0]

for df in range(1, len(df_list)):
    
    df_combine_year = pd.merge(df_combine_year, df_list[df], how = 'outer', 
                                  on = ['WISE Facility', 'Customer', 'Accounting_CustomerID'])
    print(df_combine_year.shape)
    
    df_combine_list.append(df_combine_year)
    
df_output_init = df_combine_list[-1]
df_output = df_output_init.drop_duplicates().sort_values(['WISE Facility', 'Accounting_CustomerID']).fillna(0)

In [ ]:
df_output

In [ ]:
df_sum = df_output.copy(deep = True)

df_sum['Total Bill Amount'] = df_sum.sum(axis = 1)

df_sum = df_sum.sort_values(['WISE Facility', 'Accounting_CustomerID', 'Customer'])
df_sum['WISE Facility'] = df_sum['WISE Facility'].str.title()
df_sum['Customer'] = df_sum['Customer'].str.upper()
df_sum

In [ ]:
folder_type = r'C:\Users\mazhang\Documents\Jupyter Notebook\Monthly Closed Invoice\Monthly Storage Income\2022\Raw Data'
file_type = 'Customer Type.xlsx'
df_type = pd.read_excel(os.path.join(folder_type, file_type))

df_type['Customer'] = df_type['Customer'].str.upper()
df_type

In [ ]:
df_combine = pd.merge(df_sum, df_type, how = 'left',  on = ['Customer'])
df_combine

In [ ]:
df_acc = df_combine['Account Type']
df_combine = df_combine.drop(columns = 'Account Type')
df_combine.insert(loc = 1, column = 'Account Type', value = df_acc)

df_bus = df_combine['Business Type']
df_combine = df_combine.drop(columns = 'Business Type')
df_combine.insert(loc = 2, column = 'Business Type', value = df_bus)

df_combine

In [ ]:
df_combine.to_excel("2022 Monthly Storage Income.xlsx", index = False, header = True)